# Weekly project 6
Today we will continue work from monday.
We will follow the style of last week.

Weekly project:
- You will need to implement your own k-means algorithm. You are not allowed to use the implementation in `sklearn`.
- It should be able to cluster each of the different figures.
- Extend your k-means so it finds the optimal amount of clusters.
  
## Challenge
- Implement the mean shift clustering algorithm


In [211]:
import numpy as np
import open3d as o3d
import copy
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix

def draw_labels_on_model(pcl, labels):
    cmap = plt.get_cmap("tab20")
    pcl_temp = copy.deepcopy(pcl)
    max_label = labels.max()
    colors = cmap(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcl_temp.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcl_temp])

d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=1).translate((0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=2).translate((d, -d, 0))

## apply k means on this
point_cloud = mesh.sample_points_uniformly(int(1e3))

In [212]:
# o3d.visualization.draw_geometries([point_cloud])

In [213]:
point_cloud

PointCloud with 1000 points.

In [214]:
def kmeans(point_cloud, k: int):
    
    max_iter = 10000
    centroid_threshold = 0.001
    
    cluster_centers_random = np.empty(k)
    
    points = np.asarray(point_cloud.points)
    
    cluster_centers = None
    
    # Randomly initialize k cluster centers
    random_indices = np.random.choice(points.shape[0], size=k, replace=False)   
    cluster_centers_random = points[random_indices]
    
    
    # Assign each point to its nearest cluster center
    i = 0
    while i < max_iter:
    
        if cluster_centers is None:
            cluster_centers = cluster_centers_random
            
        distances = distance_matrix(points, cluster_centers)
            
        prev_cluster_centers = cluster_centers
        
        cluster_centers = np.empty((k, points.shape[1]))
        
        closest_cluster_idx = np.argmin(distances, axis=1)
        
        # Re-compute each cluster-center as the centroid of all points assigned to each cluster
        
        for i in range(4):
            cluster_centers[i] = np.mean(points[closest_cluster_idx == i], axis=0)
        
        #Check convergence/stop criterion, otherwise repeat Steps 2-4
        
        centroid_dif = np.mean(cluster_centers - prev_cluster_centers)
        
        print(f"iteration: {i}")
        print(f"centroid_dif: {centroid_dif}")
        
        if centroid_dif < centroid_threshold:
            break
        
        i += 1
    
    return closest_cluster_idx, cluster_centers

In [215]:
clusters, recompute = kmeans(point_cloud, 6)

iteration: 3
centroid_dif: -0.17728515981807969


In [216]:
print(clusters[:4])
print(recompute)

[4 4 4 4]
[[ 2.79520138e+00  5.74891982e-01 -8.15311644e-02]
 [ 4.39963389e+00 -6.02923435e-01  5.96044963e-02]
 [ 3.10490588e+00 -3.97785219e+00 -6.54890526e-05]
 [-2.47634349e+00 -2.67659029e+00  7.36658087e-02]
 [-4.58156375e+00 -2.83561883e+00  3.90980516e-01]
 [-4.02246657e+00 -5.07818989e+00 -4.86675297e-01]]


In [217]:
draw_labels_on_model(point_cloud, clusters)

In [218]:
import numpy as np
from scipy.spatial import distance_matrix

def kmeans(point_cloud, k: int, max_iters=100, tol=1e-4):
    points = np.asarray(point_cloud.points)
    
    # Initialize cluster centers randomly from the points
    random_indices = np.random.choice(points.shape[0], size=k, replace=False)
    cluster_centers = points[random_indices]
    
    # Initialize variable to hold the recomputed cluster centers
    cluster_centers_recompute = np.empty((k, points.shape[1]))  # Shape (k, 3) for 3D points
    
    for iteration in range(max_iters):
        # Compute distances between points and cluster centers
        distances = distance_matrix(points, cluster_centers)
        
        # Assign each point to its nearest cluster center
        closest_cluster_idx = np.argmin(distances, axis=1)
        
        # Re-compute each cluster center as the centroid of all points assigned to that cluster
        for i in range(k):
            cluster_points = points[closest_cluster_idx == i]
            if len(cluster_points) > 0:  # Avoid division by zero
                cluster_centers_recompute[i] = np.mean(cluster_points, axis=0)
        
        # Check for convergence (if the cluster centers haven't changed much)
        if np.all(np.linalg.norm(cluster_centers - cluster_centers_recompute, axis=1) < tol):
            break
        
        # Update the cluster centers
        cluster_centers = cluster_centers_recompute.copy()
    
    return closest_cluster_idx, cluster_centers


In [219]:
closest_cluster_idx, cluster_centers = kmeans(point_cloud, 6)

In [220]:
draw_labels_on_model(point_cloud, closest_cluster_idx)